# Flask
Jeden z mnoha tutorialů: https://blog.miguelgrinberg.com/post/the-flask-mega-tutorial-part-i-hello-world

## Hello word
Kód nespouštět, v jupyteru flask nejde

In [ ]:
# app/__init__.py
from flask import Flask
app = Flask(__name__)
from app import routes

# app/routes.py
from app import app
@app.route('/')
def index():
    return "Hello, World!"

# spuštění
from app import app
app.run(host='0.0.0.0', port=80, debug=True)

## Výchozí stránka s požadavkem hesla
- formuláře jde vytvářet ručně (html i flask část) nebo přes *wtforms*:
     - formulář třída s definovánmi poli, lze lze automaticky kontrolovat vstupy
     - v html se pak objeví jako {{ form.password(size=32) }}{{ form.submit() }}
     - při pracování i renderování vytvořit instanci řídy:
         - při zpracování se validuje
         - při renderování se předává šabloně
- funkce flash slouží k zobrazení zprávy uživateli - místo v html šabloně
- session obsahuje proměnné daného uživatele
    - ukládám sem údaj o úspěšném přihlášení
- return
    - lze vrátit text, renderovanou šablonu, přesměrování
    - šablony jinja2 - lze jim předat proměnné (zde form), které jsou dosazeny do šablony

In [ ]:
from flask_wtf import FlaskForm
from wtforms import StringField, PasswordField, BooleanField, SubmitField
from wtforms.validators import DataRequired

class LoginForm(FlaskForm):
    password = PasswordField('Password', validators=[DataRequired()])
    submit = SubmitField('Sign In')
    
@app.route('/', methods=['GET', 'POST'])
def index():
    form = LoginForm()
    if form.validate_on_submit():
        if request.form.get('password') == app.config['PASSWORD']:
            session['logged'] = True
            flash("User logged in.")
        else:
            flash("Wrong password.")
    if session.get('logged'):
        return render_template('upload.html')
    return render_template('login.html', form=form)

Šablona login.html rozšiřuje base.html (obsahuje hlavičku, patičku), do které se vkládá do vybraného místa.
- renderuji login.html, která se doplní do base.html v pozici {% block content %}{% endblock %}
- šabloně je předána proměnná form, která obsahuje prvky formuláře, zobrazí se přes {{ form.password(size=32) }} apod.
- statické soubory (styly, skripty...) jsou ve složce static, vkládají se cesty pomocí {{ url_for('static', filename='w3.css') }}

In [ ]:
# login.html
{% extends "base.html" %}

{% block content %}
<center><h1>Welcome to Cooking App!</h1></center>
<form method="post" novalidate>{{ form.hidden_tag() }}
    <center>
    {{ form.password.label }}
    {{ form.password(size=32) }}
    {{ form.submit() }}
    </center>
</form>
{% endblock %}



# base.html
<!DOCTYPE HTML>
<html>
    <head>
        <link rel="stylesheet" href="{{ url_for('static', filename='w3.css') }}">
        ...
        <script src="{{ url_for('static', filename='script.js') }}"></script>
        <title>CookingApp</title>
    </head>
    <body>

<div class="w3-main w3-content w3-padding" style="max-width:1200px;margin-top:100px">
(zobrazení zpráv s funkce flash)
{% with messages = get_flashed_messages() %}
    {% if messages %}
        <ul class=flashes>
        {% for message in messages %}
            <li>{{ message }}</li>
        {% endfor %}
        </ul>
    {% endif %}
{% endwith %}

{% block content %}(sem se vloží obsah z dalšího souboru){% endblock %}
</div>

    </body>
</html>

## Pokud vám tohle nestačí, projďete si kódy v app pro další příklady